In [1]:
import sevenbridges as sbg
from sevenbridges.errors import SbgError
from sevenbridges.http.error_handlers import rate_limit_sleeper, maintenance_sleeper
import sys
import re
import pdb
import concurrent.futures
from requests import request
import json
config = sbg.Config(profile='turbo')
api = sbg.Api(config=config, error_handlers=[rate_limit_sleeper, maintenance_sleeper])

## Create example bash calls from workflow
Works best with non-restarted WF

In [ ]:
project = "d3b-bixu/dev-rnaseq-snv"
task_id = "f85f4ba0-7927-435e-b39a-b8c6571baa4c"
out_file = open("/Users/brownm28/Documents/2020-Apr-8_RNAseq_snv_dev/vardict_calls.tsv", "w")
# task_id = "3c20cc8e-18d7-43f2-bc2c-4a76d38a88f8"
task = api.tasks.get(task_id)
jobs = {}
temp = {}
for job in task.get_execution_details().jobs:
    if job.status == "COMPLETED":
        check = job.name.split('_')
        cmd = job.command_line
        if job.command_line == None:
            # pdb.set_trace()
            cmd = "embedded script or task retry"
            sys.stderr.write("WARN: Job " + job.name + " had null cmd\n")
        if check[-1] == "s":
            key = "_".join(check[:-2])
            if key not in temp:
                jobs[job.start_time] = {}
                jobs[job.start_time][key] = cmd
                temp[key] = 1
            else:
                temp[key] += 1
        else:
            jobs[job.start_time] = {}
            jobs[job.start_time][job.name] = cmd
out_file.write("Step\tType\tNum scatter\tCommand\n")
for rtime in sorted(jobs.keys()):
    for key in jobs[rtime]:
        rtype = "run step"
        sct = "NA"
        if key in temp and temp[key] > 1:
            rtype = "scatter"
            sct = str(temp[key])
        cmds = jobs[rtime][key].split('\n')
        for cmd in cmds:
            out_file.write(key + "\t" + rtype + "\t" + sct + "\t" + cmd + "\n")
out_file.close()


## Convert tsv to markdown table

In [ ]:
import sys

# max desired col width
max_w = 200
tsv_in = open("/Users/brownm28/Documents/2020-Apr-8_RNAseq_snv_dev/vardict_calls.tsv")
out_md = open("/Users/brownm28/Documents/2020-Apr-8_RNAseq_snv_dev/vardict_calls.md", "w")
data = []
max_len = []


for line in tsv_in:
    info = line.rstrip('\n').split('\t')
    data.append(info)
    if len(max_len) == 0:
        for item in info:
            max_len.append(len(item))
    else:
        for i in range(len(max_len)):
            if len(info[i]) > max_w:
                max_len[i] = max_w
            elif len(info[i]) > max_len[i]:
                max_len[i] = len(info[i])
# print header first
d_ct = []
for i in range(len(data[0])):
    d_ct.append(len(data[0][i]))
    out_md.write(" | " + data[0][i] + "".join([" "] * max_len[i]))
    d_ct[i] += max_len[i]
out_md.write(" |\n")
for i in range(len(data[0])):
    out_md.write(" | " + "".join(["-"] * d_ct[i]))
out_md.write(" |\n")
# pdb.set_trace()
for i in range(1, len(data), 1):
    for j in range(len(data[i])):
        d_ct = len(data[i][j]) + 2
        out_md.write(" | " + data[i][j] + "".join([" "] * max_len[j]))
        d_ct += max_len[j]
    out_md.write(" |\n")
out_md.close()

## Get run times

### Get run times by step

In [2]:
def get_job_run_time(task, phrase):
    data = []
    if re.search(phrase, task.name):
        try:
            for job in task.get_execution_details().jobs:
                if job.status != "COMPLETED":
                    sys.stderr.write("Skipping job likely killed due to spot instance kill for " + job.name + " from task " + task.id + "\n")
                else:
                    data.append([job.name, str((job.end_time-job.start_time).seconds/3600)])
            # pdb.set_trace()
            hold=1
            return task.id, task.name, str(task.price.amount), str((task.end_time - task.start_time).seconds/3600), data
        except Exception as e:
            return [e, task.id]
    else:
        return []


In [3]:
project = "kfdrc-harmonization/sd-bhjxbdqk-06"
phrase = "RNAfusion-"

tasks = api.tasks.query(project=project, status="COMPLETED").all()
actual_out = open("/Users/brownm28/Documents/2020-Apr-8_RNAseq_snv_dev/cost_est/actual_cost.txt", "w")
actual_out.write("Task name\tTask ID\tCost\tRun Time in hours\n")
step_run = open("/Users/brownm28/Documents/2020-Apr-8_RNAseq_snv_dev/cost_est/step_run_times.txt", "w")
step_run.write("Run step\tRun time in hours\n")
# for task in tasks:
#     result = get_job_run_time(task, phrase)
#     if len(result) > 0:
#         pdb.set_trace()
#         actual_out.write("\t".join(result[0:4]) + "\n")
#         for step in result[4]:
#             step_run.write("\t".join(step) + "\n")
x = 0
m = 100
with concurrent.futures.ThreadPoolExecutor(16) as executor:
    results = {executor.submit(get_job_run_time, task, phrase): task for task in tasks}
    for result in concurrent.futures.as_completed(results):
        if len(result.result()) > 2:
            if x % m == 0:
                sys.stderr.write("Processed " + str(x) + " valid tasks\n")
            actual_out.write("\t".join(result.result()[0:4]) + "\n")
            for step in result.result()[4]:
                step_run.write("\t".join(step) + "\n")
            x += 1
        elif len(result.result()) == 2:
            sys.stderr.write(str(result.result()[0]) + "\tFailed processing task ID " + result.result()[1] + "\n")
            exit(1)
actual_out.close()
step_run.close()


Skipping job likely killed due to spot instance kill for star_fusion from task 90520afe-23e8-408e-a6d1-7a4307bff6cb
Skipping job likely killed due to spot instance kill for star_fusion from task ad4166df-eb20-4403-9e62-a434fc023946
Skipping job likely killed due to spot instance kill for pizzly from task b75638c2-fdc0-4321-8320-b7b3b7b6940e
Skipping job likely killed due to spot instance kill for star_fusion from task cf2e69a6-c9b8-47d3-92a6-52c41f4cbc2a
Skipping job likely killed due to spot instance kill for star_fusion from task c011ab43-7a02-4892-981d-5f897de739ac
Skipping job likely killed due to spot instance kill for arriba_fusion from task b6276343-2100-44c2-afc7-23a45ba141cf
Skipping job likely killed due to spot instance kill for pizzly from task c9cff929-f971-41b1-90b0-bf0a3e40f2fb
Skipping job likely killed due to spot instance kill for star_fusion from task 5d9f1b03-f375-4363-bfaf-304fadb3ee6f
Skipping job likely killed due to spot instance kill for star_fusion from task 4

## Tag source files

In [7]:
header_dict = {"SampleID": "sample_id", "Sample": "OldSampleID", "ReadType": "sample_type", "Class": "Class"}
header_idx = {"SampleID": 0, "ReadType":0, "Sample": 0, "Class": 0}
fname_dict = {}

meta_manifest = open('/Users/brownm28/Documents/2020-Apr-8_RNAseq_snv_dev/public_files/mb_comb_manifest.txt')
cav_manifest = open('/Users/brownm28/Documents/2020-Apr-8_RNAseq_snv_dev/public_files/cavatica-manifest.csv')
head = next(meta_manifest)
header = head.rstrip('\n').split('\t')
for key in header_dict:
    header_idx[key] = header.index(key)
fname_idx = 1
for line in meta_manifest:
    # pdb.set_trace()
    info = line.rstrip('\n').split('\t')
    if info[header_idx['Class']] != "#N/A":
        fname = info[fname_idx]
        fname_dict[fname] = {}
        for key in header_idx:
            fname_dict[fname][header_dict[key]] = info[header_idx[key]]
meta_manifest.close()

head = next(cav_manifest)
for line in cav_manifest:
    info = line.rstrip('\n').split(',')
    if info[1] in fname_dict:
        sys.stderr.write("Tagging " + info[1] + "\n")
        cur = api.files.get(info[0])
        cur.metadata = fname_dict[info[1]]
        cur.save()
    else:
        sys.stderr.write("Skipping " + info[1] + "\n")
cav_manifest.close()

Tagging ENV-332767164.filtered.other_microbes.R1.fastq.gz
Tagging COV-20200315-P9-E01-N.filtered.sars_cov_2.R2.fastq.gz
Tagging COV-20200315-P10-H03-N.filtered.other_microbes.R1.fastq.gz
Tagging COV-20200316-P12-F08-P.filtered.other_microbes.R2.fastq.gz
Tagging COV-20200314-P9-G07-N.filtered.sars_cov_2.R2.fastq.gz
Tagging COV-20200314-P9-E09-N.filtered.other_microbes.R2.fastq.gz
Tagging COV-20200314-P9-F03-N.filtered.sars_cov_2.R2.fastq.gz
Tagging ENV-338311314.filtered.human.R2.fastq.gz
Tagging COV-20200314-P9-F07-N.filtered.sars_cov_2.R1.fastq.gz
Tagging ENV-332767798.filtered.sars_cov_2.R1.fastq.gz
Tagging COV-20200314-P9-E08-N.filtered.other_microbes.R2.fastq.gz
Tagging COV-20200315-P10-E07-P.filtered.sars_cov_2.R1.fastq.gz
Tagging COV-20200314-P9-H07-N.filtered.sars_cov_2.R2.fastq.gz
Tagging ENV-332761261.filtered.human.R1.fastq.gz
Tagging COV-20200314-P9-E04-P.filtered.other_microbes.R1.fastq.gz
Tagging ENV-332769330_S68.filtered.sars_cov_2.R1.fastq.gz
Tagging ENV-332763462.filte

Tagging COV-20200315-P10-D09-P.filtered.other_microbes.R1.fastq.gz
Tagging COV-20200315-P10-F12-N.filtered.sars_cov_2.R2.fastq.gz
Tagging ENV-338312232_S82.filtered.sars_cov_2.R2.fastq.gz
Tagging COV-20200314-P9-H07-N.filtered.other_microbes.R2.fastq.gz
Tagging COV-20200315-P9-G01-P.filtered.sars_cov_2.R2.fastq.gz
Tagging COV-20200314-P9-B03-N.filtered.sars_cov_2.R2.fastq.gz
Tagging COV-20200315-P10-B12-P.filtered.other_microbes.R2.fastq.gz
Tagging COV-20200315-P10-A09-N.filtered.other_microbes.R1.fastq.gz
Tagging COV-20200312-P2-G03-N_S39.filtered.sars_cov_2.R1.fastq.gz
Tagging ENV-332766553_S74.filtered.human.R2.fastq.gz
Tagging COV-20200316-P12-H04-N.filtered.sars_cov_2.R2.fastq.gz
Tagging COV-20200312-P2-A03-N_S35.filtered.other_microbes.R1.fastq.gz
Tagging COV-20200315-P10-F10-P.filtered.other_microbes.R1.fastq.gz
Tagging COV-20200315-P9-E02-P.filtered.other_microbes.R1.fastq.gz
Tagging COV-20200315-P10-B05-N.filtered.sars_cov_2.R2.fastq.gz
Tagging COV-20200316-P12-E09-P.filtered.

Tagging ENV-332767811_S71.filtered.sars_cov_2.R1.fastq.gz
Tagging COV-20200316-P11-H05-N.filtered.other_microbes.R2.fastq.gz
Tagging ENV-235033541.filtered.sars_cov_2.R2.fastq.gz
Tagging ENV-332766553.filtered.sars_cov_2.R1.fastq.gz
Tagging ENV-332773976_S78.filtered.other_microbes.R2.fastq.gz
Tagging COV-20200316-P11-F06-P.filtered.sars_cov_2.R1.fastq.gz
Tagging COV-20200315-P10-G04-P.filtered.other_microbes.R2.fastq.gz
Tagging ENV-332767116.filtered.other_microbes.R1.fastq.gz
Tagging ENV-332769330.filtered.other_microbes.R2.fastq.gz
Tagging ENV-332770127.filtered.sars_cov_2.R1.fastq.gz
Tagging ENV-338337295_merged_R2.fastq.gz
Tagging ENV-338314170_S84.filtered.sars_cov_2.R1.fastq.gz
Tagging COV-20200315-P10-G02-N.filtered.other_microbes.R2.fastq.gz
Tagging COV-20200316-P11-B01-CN.filtered.sars_cov_2.R2.fastq.gz
Tagging COV-20200314-P9-E07-N.filtered.sars_cov_2.R1.fastq.gz
Tagging COV-20200316-P11-H08-N.filtered.other_microbes.R2.fastq.gz
Tagging COV-20200316-P12-E05-N.filtered.other_

Tagging COV-20200316-P12-F09-N.filtered.sars_cov_2.R2.fastq.gz
Tagging COV-20200315-P10-D01-N.filtered.other_microbes.R2.fastq.gz
Tagging COV-20200312-P3-G03-P_S11.filtered.sars_cov_2.R2.fastq.gz
Tagging COV-20200314-P9-H04-N.filtered.other_microbes.R1.fastq.gz
Tagging ENV-338311314.filtered.other_microbes.R2.fastq.gz
Tagging ENV-235057493.filtered.sars_cov_2.R1.fastq.gz
Tagging COV-20200313-P4-B06-P_S23.filtered.other_microbes.R2.fastq.gz
Tagging COV-20200315-P10-B09-P.filtered.sars_cov_2.R1.fastq.gz
Tagging COV-20200312-P2-C03-P_S5.filtered.other_microbes.R1.fastq.gz
Tagging COV-20200316-P12-H04-N.filtered.sars_cov_2.R1.fastq.gz
Tagging COV-20200316-P11-E03-N.filtered.sars_cov_2.R2.fastq.gz
Tagging ENV-332769282.filtered.sars_cov_2.R2.fastq.gz
Tagging ENV-235057491.filtered.sars_cov_2.R2.fastq.gz
Tagging COV-20200315-P10-G03-N.filtered.other_microbes.R1.fastq.gz
Tagging ENV-332766382.filtered.other_microbes.R1.fastq.gz
Tagging COV-20200313-P4-B06-P_S23.filtered.sars_cov_2.R2.fastq.gz

Tagging COV-20200316-P11-H03-P.filtered.other_microbes.R2.fastq.gz
Tagging ENV-338316278.filtered.sars_cov_2.R1.fastq.gz
Tagging COV-20200312-P2-E02-P_S3.filtered.sars_cov_2.R1.fastq.gz
Tagging COV-20200316-P11-H01-N.filtered.sars_cov_2.R1.fastq.gz
Tagging COV-20200315-P10-F09-P.filtered.other_microbes.R1.fastq.gz
Tagging ENV-338319947.filtered.human.R2.fastq.gz
Tagging COV-20200316-P12-G04-P.filtered.sars_cov_2.R1.fastq.gz
Tagging COV-20200313-P5-G02-P_S25.filtered.other_microbes.R2.fastq.gz
Tagging COV-20200314-P9-C04-N.filtered.other_microbes.R1.fastq.gz
Tagging COV-20200315-P10-E04-N.filtered.other_microbes.R2.fastq.gz
Tagging COV-20200316-P12-B06-N.filtered.sars_cov_2.R1.fastq.gz
Tagging COV-20200316-P11-B02-N.filtered.sars_cov_2.R2.fastq.gz
Tagging COV-20200315-P9-D01-N.filtered.other_microbes.R1.fastq.gz
Tagging COV-20200316-P12-A04-CN.filtered.other_microbes.R2.fastq.gz
Tagging COV-20200315-P10-G07-P.filtered.other_microbes.R1.fastq.gz
Tagging ENV-338312247.filtered.human.R2.fa

...

Tagging ENV-338314157_S80.filtered.sars_cov_2.R2.fastq.gz
Tagging COV-20200316-P12-F02-P.filtered.sars_cov_2.R2.fastq.gz
Tagging COV-20200312-P2-F02-P_S4.filtered.other_microbes.R2.fastq.gz
Tagging COV-20200312-P2-D03-P_S6.filtered.sars_cov_2.R1.fastq.gz
Tagging COV-20200315-P9-H01-N.filtered.other_microbes.R1.fastq.gz
Tagging COV-20200312-P2-E03-N_S37.filtered.sars_cov_2.R1.fastq.gz
Tagging COV-20200316-P11-B09-N.filtered.sars_cov_2.R2.fastq.gz
Tagging ENV-338312232_S82.filtered.other_microbes.R2.fastq.gz
Tagging ENV-235057493.filtered.sars_cov_2.R2.fastq.gz
Tagging COV-20200316-P12-G03-P.filtered.sars_cov_2.R2.fastq.gz
Tagging COV-20200313-P5-C02-N_S58.filtered.sars_cov_2.R1.fastq.gz
Tagging COV-20200315-P10-G10-P.filtered.other_microbes.R2.fastq.gz
Tagging COV-20200315-P10-F12-N.filtered.other_microbes.R2.fastq.gz
Tagging ENV-338338122.filtered.sars_cov_2.R1.fastq.gz
Tagging COV-20200316-P11-D03-N.filtered.other_microbes.R1.fastq.gz
Tagging ENV-332770127.filtered.other_microbes.R1.f

Tagging COV-20200315-P10-D05-N.filtered.other_microbes.R2.fastq.gz
Tagging COV-20200313-P4-B02-N_S54.filtered.sars_cov_2.R1.fastq.gz
Tagging COV-20200315-P10-D11-P.filtered.sars_cov_2.R2.fastq.gz
Tagging ENV-338318855.filtered.other_microbes.R2.fastq.gz
Tagging COV-20200316-P11-A02-N.filtered.sars_cov_2.R2.fastq.gz
Tagging COV-20200316-P12-A07-CN.filtered.other_microbes.R2.fastq.gz
Tagging ENV-332767798_S72.filtered.human.R2.fastq.gz
Tagging COV-20200314-P9-G03-N.filtered.other_microbes.R1.fastq.gz
Tagging COV-20200315-P10-A02-P.filtered.other_microbes.R1.fastq.gz
Tagging COV-20200316-P12-H01-P.filtered.other_microbes.R2.fastq.gz
Tagging COV-20200316-P12-A09-P.filtered.other_microbes.R2.fastq.gz
Tagging COV-20200314-P9-G05-N.filtered.sars_cov_2.R2.fastq.gz
Tagging COV-20200316-P12-C05-N.filtered.other_microbes.R1.fastq.gz
Tagging ENV-338339731.filtered.other_microbes.R1.fastq.gz
Tagging COV-20200314-P9-G06-N.filtered.sars_cov_2.R2.fastq.gz
Tagging COV-20200315-P10-C04-P.filtered.other_

Tagging COV-20200312-P2-F01-N_S32.filtered.sars_cov_2.R1.fastq.gz
Tagging COV-20200313-P4-G01-N_S45.filtered.other_microbes.R2.fastq.gz
Tagging COV-20200314-P9-C09-N.filtered.other_microbes.R2.fastq.gz
Tagging COV-20200316-P11-C02-N.filtered.sars_cov_2.R1.fastq.gz
Tagging COV-20200314-P9-F04-P.filtered.sars_cov_2.R2.fastq.gz
Tagging ENV-332767854_S75.filtered.sars_cov_2.R1.fastq.gz
Tagging COV-20200315-P10-A03-N.filtered.sars_cov_2.R2.fastq.gz
Tagging COV-20200315-P10-E03-N.filtered.other_microbes.R1.fastq.gz
Tagging COV-20200315-P10-E06-P.filtered.other_microbes.R2.fastq.gz
Tagging ENV-235057263.filtered.human.R2.fastq.gz
Tagging COV-20200316-P11-B03-P.filtered.sars_cov_2.R2.fastq.gz
Tagging ENV-332761261.filtered.other_microbes.R2.fastq.gz
Tagging ENV-332769330_S68.filtered.other_microbes.R1.fastq.gz
Tagging ENV-332767116_S70.filtered.sars_cov_2.R2.fastq.gz
Tagging ENV-332767040.filtered.human.R2.fastq.gz
Tagging COV-20200315-P10-A03-N.filtered.sars_cov_2.R1.fastq.gz
Tagging ENV-3327

Tagging COV-20200314-P9-A08-P.filtered.other_microbes.R2.fastq.gz
Tagging ENV-332773976_S78.filtered.sars_cov_2.R1.fastq.gz
Tagging ENV-332767164.filtered.human.R2.fastq.gz
Tagging COV-20200315-P10-F05-N.filtered.sars_cov_2.R1.fastq.gz
Tagging COV-20200316-P12-H02-N.filtered.other_microbes.R1.fastq.gz
Tagging COV-20200316-P11-B06-N.filtered.sars_cov_2.R1.fastq.gz
Tagging COV-20200314-P9-F06-N.filtered.sars_cov_2.R1.fastq.gz
Tagging COV-20200311-P9-H08-N.filtered.other_microbes.R2.fastq.gz
Tagging COV-20200316-P11-F08-N.filtered.other_microbes.R1.fastq.gz
Tagging COV-20200316-P12-H07-N.filtered.other_microbes.R2.fastq.gz
Tagging ENV-235063938.filtered.human.R1.fastq.gz
Tagging COV-20200315-P10-F07-N.filtered.sars_cov_2.R1.fastq.gz
Tagging ENV-338312231.filtered.human.R2.fastq.gz
Tagging COV-20200316-P11-B02-N.filtered.sars_cov_2.R1.fastq.gz
Tagging ENV-332773976_S78.filtered.other_microbes.R1.fastq.gz
Tagging COV-20200316-P12-A06-N.filtered.sars_cov_2.R1.fastq.gz
Tagging COV-20200313-P5

Tagging COV-20200316-P11-G01-N.filtered.sars_cov_2.R2.fastq.gz
Tagging ENV-338316278_S79.filtered.human.R2.fastq.gz
Tagging COV-20200315-P10-A10-CP.filtered.sars_cov_2.R2.fastq.gz
Tagging COV-20200316-P12-H04-N.filtered.other_microbes.R2.fastq.gz
Tagging COV-20200312-P2-E05-N_S43.filtered.sars_cov_2.R1.fastq.gz
Tagging COV-20200314-P9-C05-N.filtered.other_microbes.R1.fastq.gz
Tagging COV-20200316-P12-B04-P.filtered.sars_cov_2.R2.fastq.gz
Tagging ENV-332767131_S76.filtered.other_microbes.R2.fastq.gz
Tagging COV-20200316-P11-G02-N.filtered.other_microbes.R2.fastq.gz
Tagging ENV-235072464.filtered.sars_cov_2.R1.fastq.gz
Tagging COV-20200316-P12-B01-CN.filtered.sars_cov_2.R1.fastq.gz
Tagging ENV-338316278.filtered.human.R1.fastq.gz
Tagging COV-20200315-P10-B07-N.filtered.other_microbes.R1.fastq.gz
Tagging ENV-338312247.filtered.human.R1.fastq.gz
Tagging ENV-332767040.filtered.sars_cov_2.R2.fastq.gz
Tagging COV-20200312-P2-E03-N_S37.filtered.other_microbes.R2.fastq.gz
Tagging COV-20200316-P